<img src="https://i.imgur.com/gb6B4ig.png" width="400" alt="Weights & Biases" />

<!--- @wandbcode{deepchem, v=examples} -->

# Using Weights & Biases  with DeepChem

The following tutorial is based off of Part 6 "Introduction to Graph Convolutions" of the [DeepChem tutorial series](https://github.com/deepchem/deepchem/tree/master/examples/tutorials). This notebook is a modification of the existing tutorial notebook to include examples of W&B logging.
The original tutorial can be found [here](https://github.com/deepchem/deepchem/blob/master/examples/tutorials/06_Introduction_to_Graph_Convolutions.ipynb).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/deepchem/W%26B_x_DeepChem.ipynb)

# Introduction to Graph Convolutions


In this tutorial we will learn more about "graph convolutions." These are one of the most powerful deep learning tools for working with molecular data. The reason for this is that molecules can be naturally viewed as graphs.

![Molecular Graph](https://github.com/deepchem/deepchem/blob/master/examples/tutorials/basic_graphs.gif?raw=1)

Note how standard chemical diagrams of the sort we're used to from high school lend themselves naturally to visualizing molecules as graphs. In the remainder of this tutorial, we'll dig into this relationship in significantly more detail. This will let us get a deeper understanding of how these systems work.

## Setup

To run DeepChem within Colab, you'll need to run the following installation commands. This will take about 5 minutes to run to completion and install your environment. You can of course run this tutorial locally if you prefer. In that case, don't run these cells since they will download and install Anaconda on your local machine.

In [ ]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

In [ ]:
!pip install --pre deepchem

Install Weights & Biases and log in.


In [ ]:
!pip install wandb --upgrade

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# What are Graph Convolutions?

Consider a standard convolutional neural network (CNN) of the sort commonly used to process images.  The input is a grid of pixels.  There is a vector of data values for each pixel, for example the red, green, and blue color channels.  The data passes through a series of convolutional layers.  Each layer combines the data from a pixel and its neighbors to produce a new data vector for the pixel.  Early layers detect small scale local patterns, while later layers detect larger, more abstract patterns.  Often the convolutional layers alternate with pooling layers that perform some operation such as max or min over local regions.

Graph convolutions are similar, but they operate on a graph.  They begin with a data vector for each node of the graph (for example, the chemical properties of the atom that node represents).  Convolutional and pooling layers combine information from connected nodes (for example, atoms that are bonded to each other) to produce a new data vector for each node.

# Training a GraphConvModel

Let's use the MoleculeNet suite to load the Tox21 dataset. To featurize the data in a way that graph convolutional networks can use, we set the featurizer option to `'GraphConv'`. The MoleculeNet call returns a training set, a validation set, and a test set for us to use. It also returns `tasks`, a list of the task names, and `transformers`, a list of data transformations that were applied to preprocess the dataset. (Most deep networks are quite finicky and require a set of data transformations to ensure that training proceeds stably.)

In [ ]:
import deepchem as dc

tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='GraphConv')
train_dataset, valid_dataset, test_dataset = datasets

We also need to evaluate the performance of the model while we are training. For this, we need to define a metric, a measure of model performance. `dc.metrics` holds a collection of metrics already. For this dataset, it is standard to use the ROC-AUC score, the area under the receiver operating characteristic curve (which measures the tradeoff between precision and recall). Luckily, the ROC-AUC score is already available in DeepChem. 

In [ ]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)

We will import and set up WandbLogger in order to log our information to Weights & Biases. WandbLogger by default will log training loss.


We also create a `ValidationCallback` to handle the validation scoring during training. At the interval specified, it will log the calculated metrics to Weights & Biases. 

In [ ]:
from deepchem.models.wandblogger import WandbLogger
from deepchem.models.callbacks import ValidationCallback

wandblogger = WandbLogger(project='deepchem_graphconv', entity='kshen', name="Basic")
vc_valid = ValidationCallback(valid_dataset, interval=100, metrics=[metric], transformers=transformers)

Let's now train a graph convolutional network on this dataset. DeepChem has the class `GraphConvModel` that wraps a standard graph convolutional architecture underneath the hood for user convenience. Let's instantiate an object of this class and train it on our dataset.

In [ ]:
n_tasks = len(tasks)
model = dc.models.GraphConvModel(n_tasks, mode='classification', wandb_logger=wandblogger)
model.fit(train_dataset, nb_epoch=50, callbacks=[vc_valid])
wandblogger.finish()


To get the final performance of the model, we call `model.evaluate()`.

In [ ]:
train_score = model.evaluate(train_dataset, [metric], transformers)
valid_score = model.evaluate(valid_dataset, [metric], transformers)
test_score = model.evaluate(test_dataset, [metric], transformers)
print('Training set score:', train_score)
print('Validation set score:', valid_score)
print('Test set score:', test_score)

Training set score: {'roc_auc_score': 0.9697649985438165}
Validation set score: {'roc_auc_score': 0.7386805860507888}
Test set score: {'roc_auc_score': 0.6963171073263531}


We can save our results in a wandb.Table for better visualization in the dashboard. Our table will compare the training, validation, and testing ROC-AUC score for three different models on the same dataset/task: a basic GCN, a custom GCN, and a Graph Attention Network.

In [ ]:
columns = ["run_name", "train", "val", "test"]
metrics_table = wandb.Table(columns=columns)
# Add a row for the Basic GCN
metrics_table.add_data("Basic GCN", train_score, valid_score, test_score)

The results are pretty good, and `GraphConvModel` is very easy to use. But what's going on under the hood? Could we build GraphConvModel ourselves? Of course! DeepChem provides Keras layers for all the calculations involved in a graph convolution. We are going to apply the following layers from DeepChem.

-  `GraphConv` layer: This layer implements the graph convolution. The graph convolution combines per-node feature vectures in a nonlinear fashion with the feature vectors for neighboring nodes.  This "blends" information in local neighborhoods of a graph.

- `GraphPool` layer: This layer does a max-pooling over the feature vectors of atoms in a neighborhood. You can think of this layer as analogous to a max-pooling layer for 2D convolutions but which operates on graphs instead. 

- `GraphGather`: Many graph convolutional networks manipulate feature vectors per graph-node. For a molecule for example, each node might represent an atom, and the network would manipulate atomic feature vectors that summarize the local chemistry of the atom. However, at the end of the application, we will likely want to work with a molecule level feature representation. This layer creates a graph level feature vector by combining all the node-level feature vectors.

Apart from this we are going to apply standard neural network layers such as [Dense](https://keras.io/api/layers/core_layers/dense/), [BatchNormalization](https://keras.io/api/layers/normalization_layers/batch_normalization/) and [Softmax](https://keras.io/api/layers/activation_layers/softmax/) layer.

In [ ]:
from deepchem.models.layers import GraphConv, GraphPool, GraphGather
import tensorflow as tf
import tensorflow.keras.layers as layers

batch_size = 100

class MyGraphConvModel(tf.keras.Model):

  def __init__(self):
    super(MyGraphConvModel, self).__init__()
    self.gc1 = GraphConv(128, activation_fn=tf.nn.tanh)
    self.batch_norm1 = layers.BatchNormalization()
    self.gp1 = GraphPool()

    self.gc2 = GraphConv(128, activation_fn=tf.nn.tanh)
    self.batch_norm2 = layers.BatchNormalization()
    self.gp2 = GraphPool()

    self.dense1 = layers.Dense(256, activation=tf.nn.tanh)
    self.batch_norm3 = layers.BatchNormalization()
    self.readout = GraphGather(batch_size=batch_size, activation_fn=tf.nn.tanh)

    self.dense2 = layers.Dense(n_tasks*2)
    self.logits = layers.Reshape((n_tasks, 2))
    self.softmax = layers.Softmax()

  def call(self, inputs):
    gc1_output = self.gc1(inputs)
    batch_norm1_output = self.batch_norm1(gc1_output)
    gp1_output = self.gp1([batch_norm1_output] + inputs[1:])

    gc2_output = self.gc2([gp1_output] + inputs[1:])
    batch_norm2_output = self.batch_norm1(gc2_output)
    gp2_output = self.gp2([batch_norm2_output] + inputs[1:])

    dense1_output = self.dense1(gp2_output)
    batch_norm3_output = self.batch_norm3(dense1_output)
    readout_output = self.readout([batch_norm3_output] + inputs[1:])

    logits_output = self.logits(self.dense2(readout_output))
    return self.softmax(logits_output)

We can now see more clearly what is happening.  There are two convolutional blocks, each consisting of a `GraphConv`, followed by batch normalization, followed by a `GraphPool` to do max pooling.  We finish up with a dense layer, another batch normalization, a `GraphGather` to combine the data from all the different nodes, and a final dense layer to produce the global output. 

Let's now create the DeepChem model which will be a wrapper around the Keras model that we just created. We will also specify the loss function so the model know the objective to minimize.

In [ ]:
wandblogger = WandbLogger(project='deepchem_graphconv', entity='kshen', name="Custom")
model = dc.models.KerasModel(MyGraphConvModel(), loss=dc.models.losses.CategoricalCrossEntropy(), wandb_logger=wandblogger)

What are the inputs to this model?  A graph convolution requires a complete description of each molecule, including the list of nodes (atoms) and a description of which ones are bonded to each other.  In fact, if we inspect the dataset we see that the feature array contains Python objects of type `ConvMol`.

In [ ]:
test_dataset.X[0]

Models expect arrays of numbers as their inputs, not Python objects.  We must convert the `ConvMol` objects into the particular set of arrays expected by the `GraphConv`, `GraphPool`, and `GraphGather` layers.  Fortunately, the `ConvMol` class includes the code to do this, as well as to combine all the molecules in a batch to create a single set of arrays.

The following code creates a Python generator that given a batch of data generates the lists of inputs, labels, and weights whose values are Numpy arrays. `atom_features` holds a feature vector of length 75 for each atom. The other inputs are required to support minibatching in TensorFlow. `degree_slice` is an indexing convenience that makes it easy to locate atoms from all molecules with a given degree. `membership` determines the membership of atoms in molecules (atom `i` belongs to molecule `membership[i]`). `deg_adjs` is a list that contains adjacency lists grouped by atom degree. For more details, check out the [code](https://github.com/deepchem/deepchem/blob/master/deepchem/feat/mol_graphs.py).

In [ ]:
from deepchem.metrics import to_one_hot
from deepchem.feat.mol_graphs import ConvMol
import numpy as np

def data_generator(dataset, epochs=1):
  for ind, (X_b, y_b, w_b, ids_b) in enumerate(dataset.iterbatches(batch_size, epochs,
                                                                   deterministic=False, pad_batches=True)):
    multiConvMol = ConvMol.agglomerate_mols(X_b)
    inputs = [multiConvMol.get_atom_features(), multiConvMol.deg_slice, np.array(multiConvMol.membership)]
    for i in range(1, len(multiConvMol.get_deg_adjacency_lists())):
      inputs.append(multiConvMol.get_deg_adjacency_lists()[i])
    labels = [to_one_hot(y_b.flatten(), 2).reshape(-1, n_tasks, 2)]
    weights = [w_b]
    yield (inputs, labels, weights)

Now, we can train the model using `fit_generator(generator)` which will use the generator we've defined to train the model.


In [ ]:
model.fit_generator(data_generator(train_dataset, epochs=50))
wandblogger.finish()

Now that we have trained our graph convolutional method, let's evaluate its performance. We again have to use our defined generator to evaluate model performance.

In [ ]:
train_score2 = model.evaluate_generator(data_generator(train_dataset), [metric], transformers)
valid_score2 = model.evaluate_generator(data_generator(valid_dataset), [metric], transformers)
test_score2 = model.evaluate_generator(data_generator(test_dataset), [metric], transformers)
# Add a row to the metrics table for our custom GCN
metrics_table.add_data("Custom GCN", train_score2, valid_score2, test_score2)

print('Training set score:', train_score2)
print('Validation set score:', valid_score2)
print('Test set score:', test_score2)

Training set score: {'roc_auc_score': 0.8319052187920285}
Validation set score: {'roc_auc_score': 0.6665024025036129}
Test set score: {'roc_auc_score': 0.6223467493452134}


Success! The model we've constructed behaves nearly identically to `GraphConvModel`. 

We can also use other graph models provided by Deepchem such as the [Graph Attention Model](https://deepchem.readthedocs.io/en/latest/api_reference/models.html#gatmodel).

In order to use it, we must first install DGL and DGL-LifeSci as specified in the GAT documentation. 

Creating and training a GAT model follows the exact same process as the previous two models.

In [ ]:
!pip install dgl-cu110
!pip install dgllife

In [ ]:
from deepchem.models import GATModel
featurizer = dc.feat.MolGraphConvFeaturizer()
tasks, datasets, transformers = dc.molnet.load_tox21(reload=False, featurizer=featurizer, transformers=[])
train_dataset, valid_dataset, test_dataset = datasets

wandblogger = WandbLogger(project='deepchem_graphconv', entity='kshen', name="GAT")
model = GATModel(mode='classification', n_tasks=len(tasks), batch_size=100, learning_rate=0.001, wandb_logger=wandblogger)
vc_valid = ValidationCallback(valid_dataset, interval=100, metrics=[metric], transformers=transformers)
model.fit(train_dataset, nb_epoch=50, callbacks=[vc_valid])

In [ ]:
train_score3 = model.evaluate(train_dataset, [metric], transformers)
valid_score3 = model.evaluate(valid_dataset, [metric], transformers)
test_score3 = model.evaluate(test_dataset, [metric], transformers)
# Add a row to our table for our GAT
metrics_table.add_data("Graph Attention Network", train_score3, valid_score3, test_score3)

print('Training set score:', train_score3)
print('Validation set score:', valid_score3)
print('Test set score:', test_score3)

Training set score: {'roc_auc_score': 0.8742167368065249}
Validation set score: {'roc_auc_score': 0.7489961126423411}
Test set score: {'roc_auc_score': 0.702721208818308}


In [ ]:
# Log the final table to this run
wandblogger.wandb_run.log({"Scores": metrics_table})

In [ ]:
wandblogger.finish()

# Congratulations! Time to join the Community!

Congratulations on completing this tutorial notebook! If you enjoyed working through the tutorial, and want to continue working with DeepChem, we encourage you to finish the rest of the tutorials in this series. You can also help the DeepChem community in the following ways:

## Star DeepChem on [GitHub](https://github.com/deepchem/deepchem)
This helps build awareness of the DeepChem project and the tools for open source drug discovery that we're trying to build.

## Join the DeepChem Gitter
The DeepChem [Gitter](https://gitter.im/deepchem/Lobby) hosts a number of scientists, developers, and enthusiasts interested in deep learning for the life sciences. Join the conversation!